<a href="https://colab.research.google.com/github/Ankitaagr001/ChatBot/blob/main/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')


In [ ]:
pip install nltk

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True)



True

In [ ]:
f= open('chatbot.txt', 'r', errors = 'ignore')
raw = f.read()
raw = raw.lower()

**TOKENISATION**

In [ ]:
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

**Pre**-**Processing**

In [ ]:
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

**Keyword** **matching**

In [ ]:
GREETING_INPUTS = ("hello","hi","greetings","sup","what's up","hey")
GREETING_RESPONSES = ["hi","hey","*nods*", "hi there", "hello", "I am glad", "Good morning"]
HOW_ARE_YOU_INPUTS = ("how are you", "how's it going", "how do you do")
HOW_ARE_YOU_RESPONSES = ["I'm good, thanks!", "Doing well, how about you?", "I am fine, thank you!"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

    # Check for 'how are you' inputs
    if any(phrase in sentence.lower() for phrase in HOW_ARE_YOU_INPUTS):
        return random.choice(HOW_ARE_YOU_RESPONSES)

    return None

**Weather** **Responses**

In [ ]:
pip install requests

In [ ]:
import requests

def get_weather(city):
    api_key = "061f4a6ce4b8d3a6938525942f52718f"
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}appid={api_key}&q={city}&units=metric"
    response = requests.get(complete_url)
    weather_data = response.json()
    if weather_data["cod"] != "404":
        main = weather_data["main"]
        temperature = main["temp"]
        humidity = main["humidity"]
        weather_description = weather_data["weather"][0]["description"]
        weather_report = (f"Temperature: {temperature}°C\n"
                          f"Humidity: {humidity}%\n"
                          f"Description: {weather_description.capitalize()}")
    else:
        weather_report = "Sorry, I couldn't find that city."
    return weather_report

**Generating** **Response**

In [ ]:
def response(user_response):
    robo_response = ''
    if 'weather in' in user_response:
        words = user_response.split()
        city_index = words.index('in') + 1
        city_name = words[city_index]
        robo_response += get_weather(city_name)
        return robo_response
    else:
        sent_tokens.append(user_response)
        TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
        tfidf = TfidfVec.fit_transform(sent_tokens)
        vals = cosine_similarity(tfidf[-1], tfidf)
        idx = vals.argsort()[0][-2]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[-2]
        if req_tfidf == 0:
            robo_response += "I am sorry! I don't understand you"
            return robo_response
        else:
            robo_response += sent_tokens[idx]
            return robo_response


In [ ]:
import re
def parse_weather_query(user_response):
    # Regex to find patterns like "weather in", "weather at", etc.
    match = re.search(r"weather (?:in|at|for)? ?([a-zA-Z\s]+)", user_response, re.IGNORECASE)
    if match:
        # Extract the city name from the regex group
        city_name = match.group(1).strip()
        return get_weather(city_name)
    return None
def response(user_response):
    robo_response = ''
    # Check for weather queries first
    weather_response = parse_weather_query(user_response)
    if weather_response:
        return weather_response
    else:
        sent_tokens.append(user_response)
        TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
        tfidf = TfidfVec.fit_transform(sent_tokens)
        vals = cosine_similarity(tfidf[-1], tfidf)
        idx = vals.argsort()[0][-2]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[-2]
        if req_tfidf == 0:
            robo_response += "I am sorry! I don't understand you"
        else:
            robo_response += sent_tokens[idx]
            sent_tokens.remove(user_response)  # Only remove if it was added and found
        return robo_response


In [ ]:
import re

# Assume all previous functions like response(), greeting(), get_weather() are defined
flag = True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")

while(flag):
    user_response = input()
    user_response = user_response.lower()

    if(user_response != 'bye'):
        if(user_response in ['thanks', 'thank you']):
            flag = False
            print("ROBO: You are welcome..")
        else:
            # Check if the input is a weather query
            weather_response = parse_weather_query(user_response)
            if weather_response:
                print("ROBO: " + weather_response)
            else:
                # Use the existing greeting function to check for greetings
                response_greet = greeting(user_response)
                if response_greet is not None:
                    print("ROBO: " + response_greet)
                else:
                    # Use the response function for other queries
                    response_text = response(user_response)
                    print("ROBO: ", end="")
                    print(response_text)
                    # Remove the user_response from sent_tokens only if it was added earlier
                    if 'weather' not in user_response and user_response not in sent_tokens:
                        if user_response in sent_tokens:
                            sent_tokens.remove(user_response)
    else:
        flag = False
        print("ROBO: Bye! Take care..")


ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
ROBO: Temperature: 32.05°C
Humidity: 19%
Description: Haze
ROBO: I am sorry! I don't understand you
ROBO: Temperature: 32.05°C
Humidity: 19%
Description: Haze
ROBO: I'm good, thanks!
ROBO: I'm good, thanks!
ROBO: I am sorry! I don't understand you
